# Quiz 7

In [33]:
import numpy as np
import pandas as pd
from tqdm import trange
from IPython.display import clear_output

In [34]:
def load_data(data):
    arr = pd.read_csv(data, header=None, sep=' ').to_numpy()
    return arr[:,:-1], arr[:,-1]

## Questions 13 - 15

In [94]:
class DecisionTreeNode:
    def __init__(self, i=None, s=None, theta=None):
        self.i = i
        self.s = s
        self.theta = theta

        self.false_branch = None
        self.true_branch = None

    def predict(self, X):
        y_branch = self.s * np.sign(X[:, self.i] - self.theta)

        false_inds = y_branch == -1
        X_false = X[false_inds]
        X_true = X[~false_inds]

        false_pred = self.false_branch.predict(X_false)
        true_pred = self.true_branch.predict(X_true)

        false_ptr, true_ptr = 0, 0
        y_pred = np.zeros(len(X))
        for ptr in range(len(y_branch)):
            if y_branch[ptr] == -1:
                y_pred[ptr] = false_pred[false_ptr]
                false_ptr += 1
            else:
                y_pred[ptr] = true_pred[true_ptr]
                true_ptr += 1

        return y_pred

    def size(self):
        return 1 + self.true_branch.size() + self.false_branch.size()

        
class DecisionTreeLeaf:
    def __init__(self, const=None):
        self.const = const

    def predict(self, X):
        return self.const * np.ones(len(X))

    def size(self):
        return 0

In [95]:
X_train, y_train = load_data('train.dat')

In [96]:
def impurity(y):
    return 1 - (np.sum(y == 1) / len(y)) ** 2 - (np.sum(y == -1) / len(y)) ** 2

In [157]:
def build_tree(X, y, prefix='', stop=False):
    assert(len(X.shape) == 2)
    assert(len(y.shape) == 1)

    # Check if termination criteria are met.
    if len(np.unique(X, axis=0)) == 1 or len(np.unique(y)) == 1 or stop:
        y_mode = 1 if np.sum(y == 1) * 2 >= len(y) else -1
        return DecisionTreeLeaf(y_mode)
    
    # Find the best decision stump based on Gini index and partition data
    # according to the stump.
    N = len(X)

    segments = []
    impurities = []

    for i in range(2):
        prev_impurity = -1

        inds = np.argsort(X[:, i])
        sorted_X, sorted_y = X[inds], y[inds]

        for idx in range(len(X)-1):
            this_impurity = impurity(sorted_y[:idx+1]) * (idx+1) + \
                            impurity(sorted_y[idx+1:]) * (N-idx-1)

            if this_impurity != prev_impurity:
                if segments and prev_impurity != -1:
                    segments[-1][-1] = sorted_X[idx, i]
                prev_impurity = this_impurity
                segments.append([i, sorted_X[idx, i], None])
                impurities.append(this_impurity)

        segments[-1][-1] = sorted_X[-1, i]

    assert(len(impurities) == len(segments))
    chosen = np.argsort(impurities)[0]
    i, left, right = segments[chosen]
    assert(left <= right)
    theta = (left + right) / 2

    inds = np.argsort(X[:, i])
    sorted_X, sorted_y = X[inds], y[inds]

    y_branch = np.sign(sorted_X[:, i] - theta)
    X_false, y_false = sorted_X[y_branch == -1], sorted_y[y_branch == -1]
    X_true, y_true = sorted_X[y_branch == 1], sorted_y[y_branch == 1]

    node = DecisionTreeNode(i, 1, theta)
    node.false_branch = build_tree(X_false, y_false, prefix + '|--', False)
    node.true_branch = build_tree(X_true, y_true, prefix + '|--', False)

    return node

In [144]:
clf = build_tree(X_train, y_train)
print(clf.size())

10


In [145]:
sum(clf.predict(X_train) != y_train)

0

In [146]:
X_test, y_test = load_data('test.dat')
sum(clf.predict(X_test) != y_test) / len(X_test)

0.126

## Questions 16 - 18

In [147]:
from concurrent.futures import ProcessPoolExecutor

In [158]:
class RandomForest:
    def __init__(self):
        self.trees = []

    def train(self, X, y, T, prefix=''):
#         progress = '|/-\\|'
        E_ins = 0
        for t in range(T):
            # Sample data by bagging
            bagging_idx = np.random.randint(0, len(X), len(X))
            X_bag, y_bag = X[bagging_idx].copy(), y[bagging_idx].copy()

            # Build new tree
#             if t % 50 == 0:
#                 clear_output(wait=True)
#                 print(f'{prefix} {progress[(t // 50) % len(progress)]}')
            self.trees.append(build_tree(X_bag, y_bag))
            E_ins += np.sum(self.trees[-1].predict(X) != y) / len(y)
        
        return E_ins / T

    def predict(self, X):
        preds = np.zeros((len(self.trees), len(X)))
        for i, t in enumerate(self.trees):
            preds[i] = t.predict(X.copy())
            
        return np.sign(np.sum(preds, axis=0))

In [159]:
def proc_main():
    X_train, y_train = load_data('train.dat')
    X_test, y_test = load_data('test.dat')

    E_ins, E_outs = [], []
    E_ingt = 0
    for rep in trange(10):
        rf = RandomForest()
        E_ingt += rf.train(X_train.copy(), y_train.copy(), 300)

        E_ins.append(np.sum(rf.predict(X_train) != y_train) / len(y_train))
        E_outs.append(np.sum(rf.predict(X_test) != y_test) / len(y_test))

    return E_ingt / 10, np.average(E_ins), np.average(E_outs)

In [160]:
with ProcessPoolExecutor() as exe:
    ftrs = []
    for _ in range(10):
        ftrs.append(exe.submit(proc_main))
    
    E_ingt, E_ins, E_outs = 0, 0, 0
    for ftr in ftrs:
        a, b, c = ftr.result()
        E_ingt, E_ins, E_outs = E_ingt + a, E_ins + b, E_outs + c
    
    print(E_ingt / 10, E_ins / 10, E_outs / 10)

100%|██████████| 10/10 [03:19<00:00, 19.96s/it]


0.05187333333333327 0.0 0.0743


In [ ]:
with ProcessPoolExecutor() as exe:
    ftrs = []
    for _ in range(10):
        ftrs.append(exe.submit(proc_main))
    
    E_ingt, E_ins, E_outs = 0, 0, 0
    for ftr in ftrs:
        a, b, c = ftr.result()
        E_ingt, E_ins, E_outs = E_ingt + a, E_ins + b, E_outs + c
    
    print(E_ingt / 10, E_ins / 10, E_outs / 10)

In [137]:
X_train, y_train = load_data('train.dat')
X_test, y_test = load_data('test.dat')

E_ins, E_outs = [], []

for rep in trange(100):
    rf = RandomForest()
    rf.train(X_train.copy(), y_train.copy(), 300, f'Building forest {rep+1} / {100}')

    E_ins.append(np.sum(rf.predict(X_train) != y_train) / len(y_train))
    E_outs.append(np.sum(rf.predict(X_test) != y_test) / len(y_test))

print(np.average(E_ins))
print(np.average(E_outs))

  2%|▏         | 2/100 [00:05<04:14,  2.60s/it]


KeyboardInterrupt: 

In [44]:
np.sum(np.array([[1,2],[3,4],[5,6]]), axis=0)

array([ 9, 12])

In [51]:
x = np.random.random(2)

In [61]:
print(x)

[0.6137593  0.49641294]


In [71]:
np.unique(np.array([x, x, 2*x]), axis=0)

array([[0.6137593 , 0.49641294],
       [1.22751859, 0.99282587]])